In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import cfscrape
from lxml import etree

# Initiate webdriver

In [ ]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')

### Getting products links

In [ ]:
products_collection = ['https://www.obi.de/gartenbau/terrassendielen/c/1464',
                       'https://www.obi.de/laminat-parkett-vinylboeden/laminat/c/514',
                       'https://www.obi.de/laminat-parkett-vinylboeden/musterboeden/c/2193',
                       'https://www.obi.de/laminat-parkett-vinylboeden/korkboeden/c/1174',
                       'https://www.obi.de/bauen/bauholz/c/169',
                       'https://www.obi.de/gartenbau/holzfliesen/c/1463',
                       'https://www.obi.de/moebelbau/arbeitsplatten/c/916',
                       'https://www.obi.de/moebelbau/regalboeden-moebelbauplatten/c/917']

In [ ]:
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

s = cfscrape.create_scraper()
scraper = cfscrape.create_scraper(sess=s)

In [ ]:
all_links = []
for n in range(len(products_collection)):
    
    scraped_html = scraper.get(products_collection[n], headers=header).content
    html = etree.HTML(scraped_html)
        
    # number of pages 
    path = '//*[@class="pagination-bar__link js-queryLink scrollup"]/span//text()'
    pagination = int(html.xpath(path)[-1])
    
    # how many products should be in the section
    path = '//*[@class="results-bar__variants hidden-phone hidden-tablet"]//text()'
    articles_qnt = re.findall(r'\d+(?= Artik)', html.xpath(path)[0])[0]
    articles_qnt = int(articles_qnt)
    
    print('\nSection:', n+1, '|', '#Pages:', pagination, '|',  '#Products:', articles_qnt, '|', products_collection[n])
    
    section_links = []
    for page in range(1, pagination+1):
        
        url = products_collection[n] + '?page={}'
        url = url.format(page)

        scraped_html = scraper.get(url, headers=header).content
        html = etree.HTML(scraped_html)

        page_links = []
        for link in html.xpath('//*[@class="product-wrapper wt_ignore"]//@href'):
            if re.findall(r'\/p\/\d\d\d+', link):
                page_links.append(link)
        section_links.extend(page_links)

        print('Page: ', page, '|', len(page_links), 'links extracted', '|', 'Total links (mined):', len(section_links), '|', 'Total links (website):', articles_qnt)
        if page == pagination and len(section_links) != articles_qnt:
            print('Got less links htan expected for section: ', products_collection[n] )
            break
        elif page == pagination and len(section_links) == articles_qnt:
            print('All links were extracted correctly')
            pass
        
        sleep(5)
    
    all_links.extend(section_links)


In [ ]:
len(all_links)

In [ ]:
# deduplicating links
product_links = []
for i in all_links:
    if i not in product_links:
        product_links.append(i)
len(product_links)

In [ ]:
product_links = ['https://www.obi.de' + x for x in product_links]

In [ ]:
with open('OBI-Wood-Links.pkl', 'wb') as f:
    pickle.dump(product_links, f)

# Testing options

In [ ]:
scraped_html = scraper.get(product_links[1], headers=header).content
html = etree.HTML(scraped_html)

In [ ]:
links_options = []
plain_pages = [] 

In [ ]:
%%time
# Sorting plain and oprional pages
for i in range(len(product_links)):
    link = product_links[i]
    scraped_html = scraper.get(link, headers=header).content
    html = etree.HTML(scraped_html)

    # Checking multiple options
    raw_options = html.xpath('//*[@class="selectboxes clearfix marg_t5 marg_b20"]//text()')
    if len(raw_options)>0 and link not in links_options:
        links_options.append(link)
    elif len(raw_options)==0 and link not in plain_pages:
        plain_pages.append(link)
    if i%200==0:
        print('#', i)
    
    sleep(1)

In [ ]:
len(links_options), len(plain_pages), len(links_options)+len(plain_pages)

In [ ]:
# Cheking optiopn patterns
opt_df = pd.DataFrame(columns=['Pattern'], index = range(len(links_options)))
for i in range(len(links_options)):
    link = links_options[i]
    scraped_html = scraper.get(link, headers=header).content
    html = etree.HTML(scraped_html)
    sleep(0.3)
    
    # Checking multiple options
    page_options = []
    raw_options = html.xpath('//*[@class="selectboxes clearfix marg_t5 marg_b20"]//text()')
    for opt in raw_options:
        if ':' in opt:
            page_options.append(opt)
    opt_df.loc[i] =  ' '.join(page_options)
    if i%100==0:
        print(i, ' '.join(page_options))
    
    #sleep(1)

In [ ]:
with open('OBI-links_options.pkl', 'wb') as f:
    pickle.dump(links_options, f)

In [ ]:
with open('OBI-opt_df.pkl', 'wb') as f:
    pickle.dump(opt_df, f)

In [ ]:
opt_df['Pattern'].value_counts()

In [ ]:
opt_df['Pattern'].shape

In [ ]:
opt_df = opt_df.replace('Ã¤', 'ä', regex=True)
opt_df = opt_df.replace('Ã¼', 'ü', regex=True)
opt_df = opt_df.replace('Ã¶', 'ö', regex=True)
opt_df = opt_df.replace('Ã', 'ß', regex=True)

### Getting data inside page

In [ ]:
opt_df['Pattern'].sample()

In [ ]:
i=497
browser.get(links_options[i])

### Selecting options
#### Option 1. One Parameter

In [ ]:
opt_df[(opt_df['Pattern'] == 'Farbe:') | 
       (opt_df['Pattern'] == 'Ausführung:')].shape

In [ ]:
%%time
option_single_pages_df = pd.DataFrame([])
option_single_pages_df['URL']= np.nan

start=0
indexes = opt_df[(opt_df['Pattern'] == 'Farbe:') | (opt_df['Pattern'] == 'Ausführung:')].index[start:]

for n, i in enumerate(indexes, start=start):
    if n%50==0 or n+1 == len(indexes):
        sleep(randint(1,2))
    
    print('\n')
    print('Optional product:', n+1, links_options[i])
    
    if links_options[i] in option_single_pages_df['URL'].unique():
        continue
    else:
        pass
        
    scraped_html = scraper.get(links_options[i], headers=header).content
    html = etree.HTML(scraped_html)
    
    # Getting links to all selecting options
    opt_links_raw = html.xpath('//*[@class="selectboxes clearfix marg_t5 marg_b20"]//@href') 
    opt_links = []
    for opt_link in opt_links_raw:
        if 'https://' in opt_link and opt_link not in opt_links:
            opt_links.append(opt_link)
    print('Links for product options:', len(opt_links))
   
    
    # Getting data for each option
    
    option_df = pd.DataFrame([])
    for i in range(len(opt_links)):
        sleep(0.5)

        # Getting page data
        url = opt_links[i]
        scraped_html = scraper.get(url, headers=header).content
        html = etree.HTML(scraped_html)

        # Main Table
        soup = BeautifulSoup(scraped_html, 'html.parser')
        tables = soup.findAll("table", { "class" : "c-datalist c-datalist--33" })

        tables_dict = {}
        for count, table in enumerate(tables):
            for element in table.find_all('td'):
                key = element['data-label']+'-T'+ str(count+1)
                tables_dict[key] = element.text
        main_table = pd.DataFrame(tables_dict, index=[0])
        main_table['URL'] = url

        # Title
        path = '//*[@class="h2 overview__heading"]//text()'
        title = html.xpath(path)
        title = '| '.join(title)
        main_table['Title'] = title

        # SKU
        path = '//*[@class="article-number"]//text()'
        sku = html.xpath(path)
        sku = '| '.join(sku)
        main_table['SKU'] = sku

        # Description top
        path = '//*[@class="overview__detail-list normal black"]//text()'
        descr_top = html.xpath(path)
        descr_top = '| '.join(descr_top)

        # Description bottom
        path = '//*[@data-ui-name="ads.description-text.product-details.p"]//text()'
        bottom_descr = html.xpath(path)
        bottom_descr = '| '.join(bottom_descr)

        main_table['Description'] = descr_top + ' | ' + bottom_descr

        # Price
        path = '//*[@class="span12 span-mobile12"]//text()'
        price = html.xpath(path)
        price = '| '.join(price)
        main_table['Price Raw'] = price

        # Delivery
        path = '//*[@class="button-to-card__txt--table-cell"]//text()'
        delivery = html.xpath(path)
        delivery = '| '.join(delivery)
        main_table['Delivery'] = delivery

        option_df = pd.concat([option_df, main_table], ignore_index=True)
        
        if ((i+1)%10==0 or i+1==len(all_options)) and i>0:
            print('Processed option:', i+1, '|', 'Table length:', option_df.shape[0])
            sleep(0.5)
        
    option_single_pages_df = pd.concat([option_single_pages_df, option_df], ignore_index=True)
    
    print('Products Dataframe Length:', option_single_pages_df.shape[0])
    if option_single_pages_df.shape[0] != option_single_pages_df.drop_duplicates().shape[0]:
        print('Products Dataframe Length NO DUPL:', option_single_pages_df.drop_duplicates().shape[0])

In [ ]:
missed_links = [x for x in [links_options[i] for i in indexes] if x not in option_single_pages_df['URL'].unique()]
len(missed_links)

In [ ]:
plain_pages.extend(missed_links)

#### Option 2. Two Parameters

In [ ]:
%%time
#option_pages_df = pd.DataFrame([])
#option_pages_df['URL']= np.nan

start=0
for n, i in enumerate(opt_df[opt_df['Pattern'] == 'Farbe: Ausführung:'].index[start:], start=start):
    if n%50==0 or n+1 == opt_df[opt_df['Pattern'] == 'Farbe: Ausführung:'].shape[0]:
        sleep(randint(1,2))
    
    print('\n')
    print('Optional product:', n+1, links_options[i])
    
    if links_options[i] in option_pages_df['URL'].unique():
        continue
    else:
        pass
        
    scraped_html = scraper.get(links_options[i], headers=header).content
    html = etree.HTML(scraped_html)
    
    # Getting links to all selecting options
    opt_links_raw = html.xpath('//*[@data-ui-name="ads.variants.color.enabled"]//@href') 
    opt_links = []
    for opt_link in opt_links_raw:
        if 'https://' in opt_link and opt_link not in opt_links:
            opt_links.append(opt_link)
    
    all_options = []
    all_options.extend(opt_links)
    for i, opt_link in enumerate(opt_links):
        if (i+1)%10==0 or i+1 == len(opt_links):
            print(i+1, 'out of', len(opt_links), 'main options', '|', 'Links for product options:', len(all_options))

        scraped_html = scraper.get(opt_link, headers=header).content
        html = etree.HTML(scraped_html)
        child_links = html.xpath('//*[@class="selectbox child3"]//@href')
        
        sleep(1)
        for child_link in child_links:
            if child_link not in all_options and 'https://' in child_link:
                all_options.append(child_link)
    
    # Getting data for each option
    
    option_df = pd.DataFrame([])
    for i in range(len(all_options)):
        sleep(0.5)

        # Getting page data
        url = all_options[i]
        scraped_html = scraper.get(url, headers=header).content
        html = etree.HTML(scraped_html)

        # Main Table
        soup = BeautifulSoup(scraped_html, 'html.parser')
        tables = soup.findAll("table", { "class" : "c-datalist c-datalist--33" })

        tables_dict = {}
        for count, table in enumerate(tables):
            for element in table.find_all('td'):
                key = element['data-label']+'-T'+ str(count+1)
                tables_dict[key] = element.text
        main_table = pd.DataFrame(tables_dict, index=[0])
        main_table['URL'] = url

        # Title
        path = '//*[@class="h2 overview__heading"]//text()'
        title = html.xpath(path)
        title = '| '.join(title)
        main_table['Title'] = title

        # SKU
        path = '//*[@class="article-number"]//text()'
        sku = html.xpath(path)
        sku = '| '.join(sku)
        main_table['SKU'] = sku

        # Description top
        path = '//*[@class="overview__detail-list normal black"]//text()'
        descr_top = html.xpath(path)
        descr_top = '| '.join(descr_top)

        # Description bottom
        path = '//*[@data-ui-name="ads.description-text.product-details.p"]//text()'
        bottom_descr = html.xpath(path)
        bottom_descr = '| '.join(bottom_descr)

        main_table['Description'] = descr_top + ' | ' + bottom_descr

        # Price
        path = '//*[@class="span12 span-mobile12"]//text()'
        price = html.xpath(path)
        price = '| '.join(price)
        main_table['Price Raw'] = price

        # Delivery
        path = '//*[@class="button-to-card__txt--table-cell"]//text()'
        delivery = html.xpath(path)
        delivery = '| '.join(delivery)
        main_table['Delivery'] = delivery

        option_df = pd.concat([option_df, main_table], ignore_index=True)
        
        if ((i+1)%10==0 or i+1==len(all_options)) and i>0:
            print('Processed option:', i+1, '|', 'Table length:', option_df.shape[0])
            sleep(0.5)
        
    option_pages_df = pd.concat([option_pages_df, option_df], ignore_index=True)
    
    print('Products Dataframe Length:', option_pages_df.shape[0])
    if option_pages_df.shape[0] != option_pages_df.drop_duplicates().shape[0]:
        print('Products Dataframe Length NO DUPL:', option_pages_df.drop_duplicates().shape[0])

In [ ]:
idx = opt_df[opt_df['Pattern'] == 'Farbe: Ausführung:'].index.to_list()
missed_links = [x for x in [links_options[i] for i in idx] if x not in option_pages_df['URL'].unique()]
len(missed_links)

In [ ]:
plain_pages.extend(missed_links)

In [ ]:
option_pages_df.shape

In [ ]:
option_pages_df = option_pages_df.drop_duplicates()
option_pages_df.shape

## Getting data for the pages without selecting options

### Looping through pages

In [ ]:
plain_pages_df = pd.DataFrame([])
for i in range(len(plain_pages)):
    
    if i%100==0:
        print('#', i, '|', 'Table length:', plain_pages_df.shape[0])
        sleep(randint(1,4))
    sleep(1)
    
    # Getting page data
    scraped_html = scraper.get(plain_pages[i], headers=header).content
    html = etree.HTML(scraped_html)
    
    # Main Table
    soup = BeautifulSoup(scraped_html, 'html.parser')
    tables = soup.findAll("table", { "class" : "c-datalist c-datalist--33" })
    
    tables_dict = {}
    for i, table in enumerate(tables):
        for element in table.find_all('td'):
            key = element['data-label']+'-T'+ str(i+1)
            tables_dict[key] = element.text
    main_table = pd.DataFrame(tables_dict, index=[0])
    main_table['URL'] = plain_pages[i]
    
    # Title
    path = '//*[@class="h2 overview__heading"]//text()'
    title = html.xpath(path)
    title = '| '.join(title)
    main_table['Title'] = title
    
    # SKU
    path = '//*[@class="article-number"]//text()'
    sku = html.xpath(path)
    sku = '| '.join(sku)
    main_table['SKU'] = sku
    
    # Description top
    path = '//*[@class="overview__detail-list normal black"]//text()'
    descr_top = html.xpath(path)
    descr_top = '| '.join(descr_top)
    
    # Description bottom
    path = '//*[@data-ui-name="ads.description-text.product-details.p"]//text()'
    bottom_descr = html.xpath(path)
    bottom_descr = '| '.join(bottom_descr)
    
    main_table['Description'] = descr_top + ' | ' + bottom_descr
    
    # Price
    path = '//*[@class="span12 span-mobile12"]//text()'
    price = html.xpath(path)
    price = '| '.join(price)
    main_table['Price Raw'] = price
    
    # Delivery
    path = '//*[@class="button-to-card__txt--table-cell"]//text()' #button-to-card row-fluid disabled
    delivery = html.xpath(path)
    delivery = '| '.join(delivery)
    main_table['Delivery'] = delivery
    
    plain_pages_df = pd.concat([plain_pages_df, main_table], ignore_index=True)

In [ ]:
plain_pages_df.shape

In [ ]:
with open('OBI-Plain-Pages.pkl', 'wb') as f:
    pickle.dump(plain_pages_df, f)

In [ ]:
for i in range(len(plain_pages)):
    plain_pages_df['URL'].loc[i] = plain_pages[i]

# Processing data

In [ ]:
products = pd.concat([option_single_pages_df, option_pages_df, plain_pages_df], ignore_index=True)
products.shape

### Images

In [ ]:
images_df = pd.DataFrame({'Images':np.nan, 'URL':products['URL'].unique()}, index=range(products['URL'].nunique()))
images_df.shape

In [ ]:
%%time 
for idx in images_df.index:
    try:
        url = images_df['URL'].loc[idx]
        browser.get(url)
        sleep(0.5)

        # How many images in gallery
        path = '//*[contains(@class, "ads-previewslider__item slick-slide")]/a'
        els = browser.find_elements_by_xpath(path)

        # Click to expand the image
        path = '//*[@class="ads-slider__zoom-text"]'
        browser.find_elements_by_xpath(path)[0].click()
        sleep(0.3)

    except:
        continue
    
    
    img_urls = []  
    if len(els)>1:

        for _ in range(len(els)-1):
            # Sliding gallery
            try:
                path = '//*[contains(@class, "ads-slider__slick slick-initialized slick-slider")]/button'
                browser.find_elements_by_xpath(path)[1].click()
                sleep(0.7)

                # Get the image
                path = '//*[@class="pinch-zoom-container"]/img'
                els = browser.find_elements_by_xpath(path)
                for el in els:
                    img = el.get_attribute('src')
                    if img not in img_urls and 'blind.gif' not in img:
                        img_urls.append(img)
            except:
                pass
    else:
        try:
            # Get the image
            path = '//*[@class="pinch-zoom-container"]/img'
            els = browser.find_elements_by_xpath(path)
            for el in els:
                img = el.get_attribute('src')
                if img not in img_urls and 'blind.gif' not in img:
                    img_urls.append(img)
        except:
            continue


    print('\n#', idx, '; '.join(img_urls))
    images_df['Images'].loc[idx]= '; '.join(img_urls)

    if idx%50==0 and idx>0:
        sleep(randint(1,10))


In [ ]:
images_df['Images'] = images_df['Images'].apply(lambda x: np.nan if x=='' else x)
products = pd.merge(products, images_df, on='URL', how='left')

#### Deduplication

In [ ]:
products = products[~products.duplicated(keep='last')]
products.shape

#### Improving text

In [ ]:
products = products.replace('Ã¤', 'ä', regex=True)
products = products.replace('Ã¼', 'ü', regex=True)
products = products.replace('Ã¶', 'ö', regex=True)
products = products.replace('Ã', 'ß', regex=True)
products = products.replace('â¬', '€', regex=True)

In [ ]:
with open('OBI-Wood-Products-Raw.pkl', 'wb') as f:
    pickle.dump(products, f)

#### Price/Delivery

In [ ]:
test = products.copy()

In [ ]:
test['Price'] = test['Price Raw'].str.replace(r'\| +\| +','| ').str.replace(r'^\D+','')
test['Price'] = test['Price'].str.extract(r'(.+?(?=\|))')[0]
test['Price'] = test['Price'].str.replace(r'\,','\.').str.replace(r'[^\.\d]|\.00','').astype(float)

#### Delivery and Order Online

In [ ]:
test['Delivery Status'] = test['Delivery'].str.extract(r'(ca\. \d+\-\d+ .+?(?=\|))')[0]
pat = test['Delivery'].str.extract(r'(sofort lieferbar .+?(?=\|))')[0]
test['Delivery Status'] = test['Delivery Status'].fillna(pat)
test['Delivery Yes/No'] = test['Delivery Status'].str.replace(r'.+','Yes').fillna('No')

#### SKU

In [ ]:
test['SKU'] = test['SKU'].str.replace(r'\D','')
test['SKU'].sample(2)

#### Drop missed columns

In [ ]:
missed_cols = pd.DataFrame((test.isna().sum()/test.shape[0]*100), columns=['Missed'])
cols_del = missed_cols[missed_cols['Missed']>96].index.to_list()
len(cols_del)

In [ ]:
test = test.drop(columns=cols_del)
test.shape

### Dimentions

#### Unifying to mm

In [ ]:
test['Title-Dims'] = test['Title'].str.replace(r'^.+?(?=\d)','')

In [ ]:
for i in test.index:
    if len(re.findall(r'(?<=\d) m|(?<=\d)m|(?<=\d) cm|(?<=\d)cm', test['Title-Dims'].loc[i]))==0:
        test['Title-Dims'].loc[i] = np.nan

In [ ]:
test['Title-Dims'] = test['Title-Dims'].str.replace(r'[^cmx\d\,\.]','').str.replace(r'^\D+?(?=\d)','')
test['Title-Dims'] = test['Title-Dims'].str.replace(r'cm[^x]','cm').str.replace(r'mm[^x]','mm')
test['Title-Dims'] = test['Title-Dims'].str.replace(r'cm[^x]','cm').str.replace(r'mm[^x]','mm')

In [ ]:
test['Header-Dims-Mult'] = test['Title-Dims']
for i in test.index:
    if 'cm' in test['Header-Dims-Mult'].fillna('').loc[i]:
        test['Header-Dims-Mult'].loc[i] = 10

In [ ]:
for i in test.index:
    if 'mm' in str(test['Header-Dims-Mult'].fillna('').loc[i]):
        test['Header-Dims-Mult'].loc[i] = 1

In [ ]:
header_dims = test['Title-Dims'].str.split('x', expand=True)
header_dims.isna().sum()/header_dims.shape[0]*100

In [ ]:
header_dims = header_dims.drop(columns=3).rename(columns={0:'H-Dim1', 1:'H-Dim2', 2:'H-Dim3'})
for col in ['H-Dim1', 'H-Dim2', 'H-Dim3']:
    header_dims[col] = header_dims[col].str.replace(r'\,','.').str.replace(r'[^\.\d]','')
    header_dims[col] = header_dims[col].apply(lambda x: np.nan if x=='' or x ==' '  else x)
   # header_dims[col] = header_dims[col].astype(float)
    #

In [ ]:
test = pd.concat([test, header_dims], axis=1)

### Checking missed dims

In [ ]:
dims_cols = ['Stärke-T1', 'Breite-T1', 'Länge-T1','Höhe-T2','Breite-T2','Tiefe-T2','H-Dim1','H-Dim2','H-Dim3']
test[test['Stärke-T1'].isna()][dims_cols].sample(2)

### Getting Multipliers

In [ ]:
cols = ['Stärke-T1','Breite-T1','Länge-T1','Höhe-T2','Breite-T2','Tiefe-T2']
for col in cols:
    mult_col = col + '-Mult'
    mm_col = col + '-mm'
    test[mult_col] = 1
    
    test[col] = test[col].astype(str)
    for i in test.index:
        if 'cm' in test[col].loc[i]:
            test[mult_col].loc[i] = 10
            continue

        elif 'mm' in test[col].loc[i]:
            test[mult_col].loc[i] = 1
            continue

        elif 'm' in test[col].loc[i]:
            test[mult_col].loc[i] = 1000
    test[mult_col] = test[mult_col].astype(float)
    test[mm_col] = test[col].str.replace(r'\,','.').str.replace(r'[^\.\d]','')
    test[mm_col] = test[mm_col].apply(lambda x: np.nan if x =='' or x==' ' else x).astype(float)
    
    test[mm_col] = test[mm_col]*test[mult_col]

In [ ]:
new_dims_cols = ['Stärke-T1','Stärke-T1-mm', 
                 'Breite-T1', 'Breite-T1-mm',
                 'Länge-T1','Länge-T1-mm',
                 'Höhe-T2','Höhe-T2-mm',
                 'Breite-T2','Breite-T2-mm',
                 'Tiefe-T2','Tiefe-T2-mm',
                 'H-Dim1','H-Dim2','H-Dim3','Header-Dims-Mult']
test[new_dims_cols].sample(1)

In [ ]:
dims_test = test[new_dims_cols].copy()

In [ ]:
cols=['Stärke-T1-mm','Breite-T1-mm','Länge-T1-mm']
sec_cols = ['Tiefe-T2-mm','Breite-T2-mm','Höhe-T2-mm']
for col in cols:
    vals=[]
    for sec_col in sec_cols: 
        val = dims_test[(~dims_test[col].isna()) &
                          (dims_test[col] - dims_test[sec_col]) == 0].shape[0]
        vals.append(val)
    print(vals)
    print(vals.index(max(vals)))
    print(col, sec_cols[vals.index(max(vals))])

In [ ]:
test['Stärke-T1-mm'] = test['Stärke-T1-mm'].fillna(test['Tiefe-T2-mm'])
test['Länge-T1-mm'] = test['Länge-T1-mm'].fillna(test['Höhe-T2-mm'])
test['Breite-T1-mm'] = test['Breite-T1-mm'].fillna(test['Breite-T2-mm'])

In [ ]:
test[(test['Stärke-T1-mm'].isna()) & 
     (test['Länge-T1-mm'].isna()) & 
     (test['Breite-T1-mm'].isna()) &
     (~test['H-Dim1'].isna())]['Title'].shape[0]/test.shape[0]*100

### Missed Dimentions

In [ ]:
test[(test['Stärke-T1-mm'].isna()) & 
     (test['Länge-T1-mm'].isna()) & 
     (test['Breite-T1-mm'].isna())]['Title'].shape[0]/test.shape[0]*100

In [ ]:
test[(test['Stärke-T1-mm'].isna()) & 
     (test['Länge-T1-mm'].isna()) & 
     (test['Breite-T1-mm'].isna())].shape#['Title'].value_counts()

# *

# Combine all columns to mine text form

In [ ]:
test['Combined'] = ''
for col in test.columns[1:]:
    test['Combined'] = test['Combined'] + '; ' + test[col].astype(str)

In [ ]:
test['Combined'] = test['Combined'].str.replace(r'\; nan','').str.replace(r'^; ','')

### Surface Quality

In [ ]:
quality = ['A-Sortierung', 'B-Sortierung', 'C-Sortierung', 'A/B', 'B/C','Kl. A', '1.Wahl Sortierung', 'Altholz']

In [ ]:
test['Combined Quality'] = ''
for i in test.index:
    test['Combined Quality'].loc[i] = []
    for qual in quality:
        pat = r' {} |^{} | {}$| {}\;'.format(qual.lower(), qual.lower(), qual.lower(), qual.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and qual.lower() not in ', '.join(test['Combined Quality'].loc[i]):
            test['Combined Quality'].loc[i].append(qual)
            
    try:
        test['Combined Quality'].loc[i] = ', '.join(test['Combined Quality'].loc[i])
    except:
        pass
test['Combined Quality'] =  test['Combined Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Combined Quality'].value_counts()

### Sorting Class

In [ ]:
sort_classes = ['S7', 'S10', 'S13', 'C16M', 'C24M', 'C30M', 'D30', 'D35', 'D40',
               'D60', 'D70', 'geringe tragfähigkeit', 'mittlere tragfähigkeit',
               'hohe tragfähigkeit']

In [ ]:
test['Combined Sorting Class'] = ''
for i in test.index:
    test['Combined Sorting Class'].loc[i] = []
    for sort_class in sort_classes:
        pat = r' {} |^{} | {}$| {}\;'.format(sort_class.lower(), sort_class.lower(), sort_class.lower(), sort_class.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and sort_class.lower() not in ', '.join(test['Combined Sorting Class'].loc[i]):
            test['Combined Sorting Class'].loc[i].append(sort_class)
       
    try:
        test['Combined Sorting Class'].loc[i] = ', '.join(test['Combined Sorting Class'].loc[i])
    except:
        pass
test['Combined Sorting Class'] =  test['Combined Sorting Class'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Combined Sorting Class'].value_counts()

## Main parameters

### Wood Types

In [ ]:
woods = ['Ahorn', 'Tanne', 'Sperrholz', 'Nussbaum', 'Erle',
        'Bongossi (Azobé)', 'Birke', 'Buche', 'Douglasie',
        'Edelkastanie', 'Eiche', 'Elsbeere', 'Erle', 'Esche', 'Fichte',
        'Kiefer', 'Kirschbaum', 'Lärche', 'Linde', 'Mahagoni', 'Teak',
        'Pappel', 'Robinie', 'Tanne', 'Ulme', 'Walnuss', 'Weide', 'Birne',
        'Fichte/Tanne', 'Fichte/Kiefer']

In [ ]:
test['Meta Wood'] = ''
for i in test.index:
    for wood in woods:
        pat = r' {} |^{} | {}$'.format(wood.lower(), wood.lower(), wood.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0:
            test['Meta Wood'].loc[i] = wood

In [ ]:
test['Meta Wood'] = test['Meta Wood'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [ ]:
test['Wood Type'] = test['Material-T1'].fillna(test['Holzart-T1']).fillna(test['Holzart-Dekor-T1']).fillna(test['Meta Wood'])

#### Cheking wrong typs and woods

In [ ]:
wrong_wood = ['Holzwerkstoff (MDF)', 'Glas', 'Aluminium', 'Metall', 'Granit', 'Polymere', 'Granito',
              'Kunststoff','Laminiert','Gipskarton, Furniert','PVC','Beschichtet', 'Melaminbeschichtet' ,
              'Furniert','PVC-freie Werkstoffmischung',  'Foliert',  'Laminiert', 'Polymere','Polycarbonat',
              'Polyvinylchlorid (PVC)','Gipskarton','German Compact Composite (GCC)','Bandstahl','Foliert',
              'Faseprofil','Metall, Kunststoff', 'Textil, Kunststoff','Rotbuche','Multiplex',
              'Qualitätsspanplatte mit HPL-Beschichtung Profil aus Hart-PVC','Holz-Kunststoff-Verbundwerkstoffe (WPC)'
              ,'Beschichtete Flachpressplatte', 'Hart-PVC-Leiste','Lackiert','WPC', 
              'Weich-PVC','Unbehandelt','Kunststoffbeschichtet','Kunststoffummantelter Hartfaserkern',
               'Stahl','Beschichtete Flachpressplatte, Hart-PVC-Leiste','Beidseitig mit HPL beschichtete Spanplatte',
              'Polycarbonat', 'German Compact Composite (GCC)', 
                'Foliert', 'WPC', 'Multiplex', 'Beschichtet', 'Stahl']


In [ ]:
# Percentage of wrong wood types
test[(test['Wood Type'].isin(wrong_wood))].shape[0]/test.shape[0]*100

In [ ]:
test[(test['Wood Type'].isin(wrong_wood))]['Wood Type'].value_counts()

In [ ]:
test = test[(~test['Wood Type'].isin(wrong_wood))]
test.shape

In [ ]:
test['Wood Type'] = test['Wood Type'].str.lower()

### Surface

In [ ]:
strong_surfaces = ['geschliffen', 'gehobelt', 'gefast',
                   'sägerau', 'geriffelt', 'glatt gehobelt', 'glatte', 'gefräst',
                   'gebürstet', 'strukturiert', '1-Seitig gehobelt',
                   '2-Seitig gehobelt', '3-Seitig gehobelt', '4-Seitig gehobelt',
                   'genutet', 'unbehandelt', 'naturbelassen', 
                   'deckend', 'lasiert', 'vorgeölt', 'kesseldruckimprägniert (KDI)',
                   'hitzebehandelt', 'thermobehandelt', 'wachsen', 'imprägniert',
                   'Scharfkantig']

weak_surfaces = ['roh', 'keine']

In [ ]:
test['Combined Surface Treat'] = ''
for i in test.index:
    test['Combined Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in test['Combined'].loc[i].lower() and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    try:
        test['Combined Surface Treat'].loc[i] = ', '.join(test['Combined Surface Treat'].loc[i])
    except:
        pass

In [ ]:
test['Combined Surface Treat'] = test['Combined Surface Treat'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [ ]:
test['Surface'] = test['Oberflächenbehandlung-T1'].fillna(test['Oberfläche-T1']).fillna(test['Combined Surface Treat'])

### Drying method

In [ ]:
dry_method = ['technischer trocknung','technisch getrocknet', 'kammer getrocknet',
               'kammertrocken', 'AD (luftgetrocknet)',
               'KD (Künstlich getrocknet)']

In [ ]:
test['Combined Drying Method'] = ''
for i in test.index:
    test['Combined Drying Method'].loc[i] = []
    for method in dry_method:
        if method.lower() in test['Combined'].loc[i].lower() and method.lower() not in ', '.join(test['Combined Drying Method'].loc[i]):
            test['Combined Drying Method'].loc[i].append(method)
        
    try:
        test['Combined Drying Method'].loc[i] = ', '.join(test['Combined Drying Method'].loc[i])
    except:
        pass

In [ ]:
test['Combined Drying Method'] =  test['Combined Drying Method'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [ ]:
test['Combined Drying Method'].value_counts()

### Keywords

In [ ]:
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
keywords = g2d.download(gfile=spreadsheet_key, wks_name = 'Keywords', credentials=credentials, col_names=True, row_names=True)

In [ ]:
rename_cols = {'Produktart':'Product Type', 
               'Holztyp':'Wood Type',
               'Qualität':'Quality',
               'Oberflächenqualität':'Surface Quality',
               'Sortierklasse':'Sorting Class',
               'Trocknung':'Drying Method'}
keywords = keywords.rename(columns=rename_cols)

In [ ]:
keywords.head(1)

In [ ]:
for col in keywords.columns:
    keywords[col] = keywords[col].apply(lambda x: np.nan if x == '' or x == ' ' or x== '-777' else x)

In [ ]:
#keywords['Wood Type'].loc[31] = 'Holz'

### Updating Product types

In [ ]:
test['Product Type'] = test['Art-T1'].copy()

In [ ]:
test['Product Type New'] = np.nan
test['Product Type New'].loc[0] = ''

In [ ]:
for i in matched_df.index:
    test['Product Type New'].loc[i] = []
    for key_word in keywords['Product Type'].unique():
        if key_word.lower() in test['Combined'].loc[i].lower():
            test['Product Type New'].loc[i].append(key_word)
    test['Product Type New'].loc[i] = ', '.join(test['Product Type New'].loc[i])

In [ ]:
test['Product Type'] = test['Product Type'].fillna(test['Product Type New'])

In [ ]:
wrong_products = ['Befestigungssysteme', 'Zubehör']

In [ ]:
test = test[~test['Product Type'].isin(wrong_products)]
test.shape

In [ ]:
#test['Product Type'].value_counts()

### Matching keywords

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
ren_cols = {#'Product Type New':'Product Type',
           'Combined Sorting Class':'Sorting Class',
           'Combined Quality':'Quality',
            'Combined Drying Method':'Drying Method',
            'Combined Surface Treat':'Surface Treatment',
            'Combined Surface Quality':'Surface Quality'}

In [ ]:
test = test.rename(columns=ren_cols)

In [ ]:
main_cols = ['URL','SKU','Product Type','Wood Type','Surface Treatment',
             'Drying Method','Sorting Class','Quality','Combined'
            #'Header','Surface Quality',
            ]

In [ ]:
test['Matched Keywords'] = np.nan
test['Other Keywords'] = np.nan
test = test.fillna('-777')
keywords = keywords.fillna('-999')

In [ ]:
%%time
def get_keywords(row):
    row['Matched Keywords'] = {}
    row['Other Keywords'] = {}
    
    matched_cols=[]
    for col in ['Product Type','Wood Type','Surface Treatment',#'Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        for key_word in keywords[col].unique():
            if fuzz.token_set_ratio(str(row[col]).lower(), key_word.lower()) > 90 or key_word.lower() in str(row[col]):
                row['Matched Keywords'][row[col]] = key_word
                matched_cols.append(col)
    
    if len(matched_cols)==0:
        row['Other Keywords'] = np.nan
        row['Matched Keywords'] = np.nan
    
    for col in ['Product Type','Wood Type','Surface Treatment',#'Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        if col not in matched_cols and len(matched_cols)>0 and row[col]!=-999:
            row['Other Keywords'][col] = row[col]
    
    return row
test = test.apply(get_keywords, axis=1)

In [ ]:
test[['Matched Keywords','Other Keywords']].isna().sum()/test.shape[0]*100

### Cleaning NaNs



In [ ]:
for col in test.columns:
    test[col] =  test[col].apply(lambda x: np.nan if x =='' or x == 'nan' or x == ' ' or x == '-777' or x == '-999' else x)

In [ ]:
test[(test['Matched Keywords'].isna())].shape

In [ ]:
test[(test['Matched Keywords'].isna()) & 
          (test['Wood Type']=='Holz')][main_cols].shape

In [ ]:
test[(~test['Matched Keywords'].isna()) #&
          ]['Product Type'].nunique()#value_counts()

### Pre-cleaning

In [ ]:
for col in test.columns:
    test[col] = test[col].apply(lambda x: np.nan if x == '' or x == ' ' or x=='-999' or x=='-777' or x=='nan'else x)

# GS

In [ ]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'

In [ ]:
sheets_clean.head(1)

In [ ]:
sheets_clean = test[~test['Matched Keywords'].isna()]
sheets_clean.shape

In [ ]:
sheets_clean = sheets_clean[(~sheets_clean.duplicated(subset='URL', keep='last'))]
sheets_clean.shape

In [ ]:
sheets_clean['Website Name'] = 'www.obi.de'

In [ ]:
ren_cols = {'Title':'Product Title',
            'Breite-T1-mm':'Width (mm)',
            'Länge-T1-mm':'Length (mm)', 
            'Stärke-T1-mm':'Thickness (mm)',
            'Price':'Price (Euro)'}
sheets_clean = sheets_clean.rename(columns = ren_cols)

cols = ['Website Name', 'URL','Product Title','Product Type', 'Wood Type', 'Width (mm)', 'Length (mm)', 'Thickness (mm)', 
        'Price (Euro)','Delivery Yes/No','Quality', 'Surface Treatment','Drying Method', #'Surface Quality',
        'Sorting Class','SKU','Delivery Status' ]

sheets_clean = sheets_clean[cols].rename(columns = ren_cols)

In [ ]:
sheets_clean = sheets_clean[~sheets_clean['Product Type'].isin(wrong_products)]

In [ ]:
sheets_clean[(sheets_clean['Width (mm)'].isna()) &
             (sheets_clean['Length (mm)'].isna()) &
             (sheets_clean['Thickness (mm)'].isna())].shape[0]/sheets_clean.shape[0]*100

In [ ]:
d2g.upload(sheets_clean.fillna('-'),
           spreadsheet_key,
           'OBI Clean Data',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)